In [1]:
import win32com.client

In [2]:
import pythoncom

In [3]:
import os, sys
import pandas as pd
from pandas import DataFrame, Series, Panel

### xingAPI COM 개발가이드.pdf 참조<br />
윈도우는 나한테 메세지를 주는 '메세지 전달자'! xingAPI를 깔면 증권사 서버랑 연결되면서 주고 받고 하게 되요.xingAPI는 누가 데이터를 요청했는지 실체를 모르거든요, 그냥 데이터를 가져오면 윈도우가 그걸 여기 쥬피터노트북으로 가져와주는 거에요.<br />
### xingAPI가 제공하는 건 크게 3가지<br />
+ Session -> Event 
<br />세션은 로그인에 관한 COM, 로그인이 됬다고 하고 메세지(이벤트)를 윈도우에 넘겨주는 방식, 주체는 세션, 넘겨주는 건 이벤트
+ Quary -> Event
+ Real -> Event

In [4]:
# 이 클래스(메세지)를 넘겨줘
# OnLogin은 로그인 됬으면 됬다, (로그아웃, Disconnect)
class XASessionEvents:
    상태 = False
    
    def OnLogin(self,code,msg):
        print('OnLogin : ',code,msg)
        XASessionEvents.상태 = True
        
    def OnLogout(self):
        print('-------------------')
        pass
    
    def OnDisconnect(self):
        print('===================')
        pass 

In [5]:
# 로그인을 하려면 정보가 필요하잖아요, 일단 url이 필요해요
# 요함수를 이용해서 내 프로그램이랑 세션(COM)을 연결해줘
# Session(DLL)-(윈도우가 위 클래스 형태로 Msg(Session이란 클래스의 복사본)를 들고)->나한테
# 로그인이 끝나고 오면 OnLogin을 부르면서 XASessionEvents에서 상태가 True가 되는데, 그 때까지 기다려야해

def Login(url='demo-ebestsec.co.kr', port=200001, svrtype=0, id ='userid',pwd='password',cert='공인인증 비밀번호'):
    session = win32com.client.DispatchWithEvents('XA_Session.XASession',XASessionEvents)
    result = session.ConnectServer(url,port)
    
    if not result:
        nErrCode = session.GetLastError()
        strErrMsg = session.GetErrorMessage(nErrCode)
        return (False, nErrCode, strErrMsg, None, session)
    
    session.Login(id, pwd, cert, svrtype, 0)
    
    while XASessionEvents.상태 == false:
        pythoncom.PumpWaitingMessage()
        
    계좌 = []
    계좌수 = session.GetAccountListCount()
    
    for i in range(계좌수):
        계좌.append(session.GetAccountList(i))
        
    return (True, 0, 'OK',계좌, session)
    

In [8]:
if __name__ == '__main__':
    #계좌정보 = pd.read_csv('secret/passwords.csv')
    #주식계좌정보 = 계좌정보.query('구분 == 거래')
    #id = 주식계좌정보['사용자ID'].values[0].strip()
    #pwd = 주식계좌정보['비밀번호'].values[0].strip()
    #cert = 주식계좌정보['공인인증비밀번호'][0].strip()
    #url = 주식계좌정보['url'][0].strip()
    
    result, code, msg, 계좌, session = Login(url='demo.ebestsec.co.kr', port = 200001, svrtype=0, id='', pwd='', cert='')
    if result == False:
        sys.exit(0)
        
    for i in 계좌:
        print(i)

com_error: (-2147221005, '잘못된 클래스 문자열입니다.', None, None)